In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
# import HumanMessages for the LLM invocation
from langchain.messages import HumanMessage, SystemMessage
# import checkpointer
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig

from langgraph.graph import MessagesState

from langchain.messages import AnyMessage, RemoveMessage

from langgraph.prebuilt import ToolNode, InjectedState
from langchain.tools import tool, ToolRuntime
from langchain_community.tools.file_management.read import ReadFileTool

from langgraph.graph import StateGraph, END, START
from langgraph.types import interrupt, Command

from pydantic import BaseModel, Field
from typing import Optional

import sqlite3
import os
from dotenv import load_dotenv
load_dotenv()

c:\Users\Andrew\Documents\dkit-projects\agentic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


True

In [4]:
reflection_llm = ChatGoogleGenerativeAI(
    model=os.getenv("GOOGLE_API_MODEL"),
    temperature=0)

In [20]:
from langchain_ollama import ChatOllama
llm = ChatOllama(
    # model="granite4:350m",
    # model="qwen3:4b",
    model="granite4:latest",
    temperature=0
)

In [10]:
import sqlite3

DB_FILE = 'music.db'

@tool
def get_album_by_title(title):
    """
    Returns the album with the given title.

    Args:
        title (str): Title of the album to retrieve.

    Returns:
        tuple: Album data (number, year, album, artist, genre, subgenre, price)
    """
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    query = "SELECT * FROM music WHERE album = ?"
    c.execute(query, (title,))
    result = c.fetchone()
    conn.close()
    return result

@tool
def get_albums_by_artist(artist):
    """
    Returns all albums by the given artist.

    Args:
        artist (str): Artist name to retrieve albums for.

    Returns:
        list: List of album data (number, year, album, artist, genre, subgenre, price)
    """
    print("DEBUG: get_albums_by_artist", artist)
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    query = "SELECT * FROM music WHERE artist = ?"
    c.execute(query, (artist,))
    results = c.fetchall()
    conn.close()
    return results

@tool
def get_albums_by_year(year):
    """
    Returns all albums released in the given year.

    Args:
        year (int): Year to retrieve albums for.

    Returns:
        list: List of album data (number, year, album, artist, genre, subgenre, price)
    """
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    query = "SELECT * FROM music WHERE year = ?"
    c.execute(query, (year,))
    results = c.fetchall()
    conn.close()
    return results

@tool
def get_albums_by_genre(genre):
    """
    Returns all albums of the given genre.

    Args:
        genre (str): Genre to retrieve albums for.

    Returns:
        list: List of album data (number, year, album, artist, genre, subgenre, price)
    """
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    query = "SELECT * FROM music WHERE genre = ?"
    c.execute(query, (genre,))
    results = c.fetchall()
    conn.close()
    return results

In [11]:
class RecordShopState(MessagesState):
    email: str
    success: bool
    count: int = 0

In [12]:
def action(state: RecordShopState) -> dict:
    result = llm.bind_tools([get_album_by_title, get_albums_by_artist, get_albums_by_year, get_albums_by_genre]).invoke(state['messages'])
    return {"messages": result}


In [13]:
class ReflectionResponse(BaseModel):
    success: bool = Field(description="Indicates whether the previous action was successful.")
    correction_prompt: str = Field(description="New improved prompt to help the agent correct its response if success is False.")

In [16]:
def reflection_action(state: RecordShopState) -> dict:
    # get last message and reflect on it against specific criteria
    last_message = state['messages'][-1].content
    system_message = state['messages'][0].content
    print("RELECTION MODEL CHECKING LAST MESSAGE:")
    reflection_prompt = f"""
    You are reflecting on the output of a music album query system.  The agent has the following PROMPT:
<PROMPT>    
{system_message}
</PROMPT>

The customer's email query was:
<EMAIL>
{state['email']}
</EMAIL>


The last response from the agent was:
<RESPONSE>
{last_message}
</RESPONSE>


If the response does not meet the goal in the prompt WRITE an IMPROVED prompt, different from the original prompt, that would help the agent produce a better response next time.
The instructions must concise.  Use bullet points if helpful. 
"""
    reflection_response = reflection_llm.with_structured_output(ReflectionResponse).invoke([HumanMessage(content=reflection_prompt)])
    if reflection_response.success:
        return {"success": True}
    new_messages = [RemoveMessage(id=m.id) for m in state["messages"]] + [SystemMessage(content=reflection_response.correction_prompt)] + [HumanMessage(content=state['email'])]
    # new_messages = HumanMessage(content=reflection_response.correction_prompt)
    return {"messages": new_messages, "success": False, "count": state['count'] + 1}
    

In [17]:
graph = StateGraph(RecordShopState)
graph.add_node("action", action)
graph.add_node("tools", ToolNode([get_album_by_title, get_albums_by_artist, get_albums_by_year, get_albums_by_genre]))
graph.add_node("reflection", reflection_action)

graph.add_edge(START, "action")
# conditional edge - if tool_calls == [] for to END else to "tools"
graph.add_conditional_edges("action", 
    lambda state: state['messages'][-1].tool_calls == [],
    path_map={True: "reflection", False: "tools"}
    )
graph.add_edge("tools", "action")
graph.add_conditional_edges("reflection", lambda state: state['success'] == True or state['count'] > 10, path_map={True: END, False: "action"})

compiled_graph = graph.compile()
    

In [ ]:
compiled_graph

In [18]:
system_prompt_react = """
You are a helpful assistant in a record shop that sells albums.  
You are answering queries from emails about what albums are available or making recommendations.
Answer the queries in a friendly and informative way.  If the email includes the customer name please use it. Sign your emails in the name of Andrew.
Think step by step and call the tools as needed to get the information required to answer the query.

"""
prompt_react = """


The query is:
{query}
"""

In [21]:
import pandas as pd

df = pd.read_csv("emails.csv")
for index, row in df.iterrows():
    email = row['email']
    run = row['run']
    if run == "yes":
        messages = [SystemMessage(content=system_prompt_react),
                    HumanMessage(content=prompt_react.format(query=email))]
        response = compiled_graph.invoke(input=RecordShopState(messages=messages, success=True, count=0, email=email), config=RunnableConfig(recursion_limit=100))
        # response = main_react(email)
        df.at[index, 'response'] = response['messages'  ][-1].content
        print(response['messages'][-1].content)

DEBUG: get_albums_by_artist David Bowie
RELECTION MODEL CHECKING LAST MESSAGE:
Hi Tom,

Great news! We have several David Bowie albums available in our store. Here are a few highlights:

1. **The Rise and Fall of Ziggy Stardust and the Spiders From Mars** (1972) – A classic rock masterpiece that defined his glam persona.
2. **Hunky Dory** (1971) – One of his early works showcasing raw talent.
3. **Low** (1977) – An experimental electronic/rock fusion album.
4. **Aladdin Sane** (1973) – Another powerful glam rock release.
5. **Station to Station** (1976) – A blend of rock, funk, and soul.

Let me know if you’d like more details on any of these or need help finding a specific one!

Cheers,
Andrew


In [125]:
response['messages'][-1]

HumanMessage(content='Do you have any David Bowie albums available? Cheers Tom', additional_kwargs={}, response_metadata={}, id='a55f3aef-5b4f-4f2f-99f6-514663204db8')